## Superscalar Hash
---

In [2]:
#![feature(portable_simd)]
use std::simd::{u64x64, SimdUint};

In [3]:
fn to_vu64(vu8: &[u8]) -> Vec<u64> {
    let M = vu8.len();
    let mut vu64 = Vec::<u64>::with_capacity(1+M/8);
    let mut a = 0usize;
    loop {
        let b = (a + 8).min(M);
        let ve_raw: [u8; 8] = [vu8[a..b].to_vec(), vec![13u8; 8-(b-a)]].concat().try_into().unwrap();
        let ve = u64::from_le_bytes(ve_raw);
        vu64.push(ve);
        a += 8;
        if b >= M { break; }
    };
    vu64
}

In [4]:
fn to_vu64x64(vu64: Vec<u64>) -> Vec<u64x64> {
    let M = vu64.len();
    let mut vu64x64 = Vec::<u64x64>::with_capacity(1+M/64);
    let mut a = 0usize;
    loop {
        let b = (a + 64).min(M);
        let ve_raw: [u64; 64] = [vu64[a..b].to_vec(), vec![11u64; 64-(b-a)]].concat().try_into().unwrap();
        let ve = u64x64::from_slice(&ve_raw);
        vu64x64.push(ve);
        a += 64;
        if b >= M { break; }
    };
    vu64x64
}

In [5]:
fn hash(vu8: &[u8]) -> u64 {
    let vu64x64 = to_vu64x64(to_vu64(vu8));
    let mut a = u64x64::splat(3);
    let mut b = u64x64::splat(7);
    for ve in vu64x64.iter().cycle().take(vu64x64.len().max(128)) {
        a ^= ve;
        b += a;
        a.rotate_lanes_left::<7usize>();
        b.rotate_lanes_right::<3usize>();
        a += b;
        b ^= ve;
    }
    (a ^ b).reduce_sum()
}

In [6]:
hash(&[1;64*17])

11083620543309429344

In [7]:
hash(&[1;64*18])

9776457142791262464

In [8]:
hash(&[1;64*36])

6327708241353846848

In [9]:
hash(&[0])

10212024217786011020

In [10]:
hash(b"")

7046882911117918476

In [11]:
hash(&[1])

3990450833070917316

In [12]:
hash(&[2])

2316560593351559728

In [13]:
hash(b"test1")

4810479651047148628

In [14]:
hash(b"test2")

5005862248827653204

In [15]:
hash(b"test3")

12820864292566404180